# Multivariate linear regression


In this notebook, the multivariate linear regression will be used to predict the price of houses. The Boston house price dataset is available at https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html. The dataset includes many attributes per house and its price.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
from IPython.display import Image

In [ ]:
mlr_data = np.load('multivariate_linear_regression_dataset.npz')
X_train = mlr_data['X']
y_train = mlr_data['y']
print(X_train.shape)
print(y_train.shape)

(506, 13)
(506,)


The solution of the linear regression can be written as
\begin{align}
\mathbf{w} = \mathbf{X}^{+}\mathbf{y}\;,
\end{align}
where $\mathbf{X}^{+}$ is the pseudoinverse of $\mathbf{X}$.
Recall that $\mathbf{X}$ is a matrix where each row is one sample and $\boldsymbol{y}$
is a vector of target values.



Using the Numpy pseudoinverse function, a function will be defined to fit a linear regression model to the data.
If adding a bias, just simply augmenting the data.

In [ ]:
def fit_LinearRegression(X, y, bias=True):

    # Insert constant ones for bias weights
    if (bias):
        X = np.insert(X, 0, 1, axis=1)

    w = np.linalg.pinv(X).dot(y)
    return w


Define another function for prediction $\hat{y} = \mathbf{w}^\top\mathbf{x}$. The function below will also compute the mean square error defined as
\begin{align}
\mathcal{L}_{SE} = \frac{1}{m} \sum_{i=1}^m \|y_i - \hat{y}_i\|^2
\end{align}


In [ ]:
def predict_LinearRegression(w, X, y, bias=True):

    # Insert constant ones for bias weights

    if (bias):
        b_star = w[0]  #recall that with bias, put the bias into the first element of w (index = 0)
        w_star = w[1:]
        y_hat = X.dot(w_star) + b_star
    else:
        y_hat = X.dot(w)


    mse = np.mean(np.square(y-y_hat))
    return y_hat, mse


First fit a linear model without bias and evaluate it as follows

In [ ]:
w1 = fit_LinearRegression(X_train, y_train, bias=False)
y_hat1, mse1 = predict_LinearRegression(w1,X_train, y_train, bias=False)

Then fit a linear model with bias and evaluate it as follows

In [ ]:
w2 = fit_LinearRegression(X_train, y_train, bias=True)
y_hat2, mse2 = predict_LinearRegression(w2,X_train, y_train, bias=True)
print("Regression Error without bias:", mse1)
print("Regression Error with bias:", mse2)

Regression Error without bias: 24.166099330126492
Regression Error with bias: 21.894831181729206


The model with bias has a lower error. Now turn to check some of the values

In [ ]:
df = pandas.DataFrame({"y": y_train[:5], "without bias": y_hat1[:5], "with bias": y_hat2[:5]})
pandas.set_option('colheader_justify', 'center')
print(df)

     y   without bias  with bias
0  24.0    29.098264   30.003843
1  21.6    24.502275   25.025562
2  34.7    31.227426   30.567597
3  33.4    29.707104   28.607036
4  36.2    29.564796   27.943524


It seems that for the very first five samples, the model without bias works better.

In [ ]:
df = pandas.DataFrame({"y": y_train[5:10], "without bias": y_hat1[:5], "with bias": y_hat2[5:10]})
pandas.set_option('colheader_justify', 'center')
print(df)

     y   without bias  with bias
0  28.7    29.098264   25.256284
1  22.9    24.502275   23.001808
2  27.1    31.227426   19.535988
3  16.5    29.707104   11.523637
4  18.9    29.564796   18.920262


In some cases the model with bias does a much better job.
Some non-linear features can be added to the model. The idea folows the concept of polynomial regression, check this [Wikipedia page](https://en.wikipedia.org/wiki/Polynomial_regression) for a brief inroduction

In [ ]:
X_quad = np.concatenate((X_train, X_train**2),axis=1)
w3 = fit_LinearRegression(X_quad, y_train, bias=True)
y_hat3, mse3 = predict_LinearRegression(w3, X_quad, y_train, bias=True)
print("Regression Error with with poly-2 features:", mse3)

Regression Error with with poly-2 features: 14.24702704237634


In [ ]:
df = pandas.DataFrame({"y": y_train[:5], "linear features": y_hat2[:5], "poly-2 features": y_hat3[:5]})
pandas.set_option('colheader_justify', 'center')
print(df)

     y   linear features  poly-2 features
0  24.0     30.003843        28.340697   
1  21.6     25.025562        23.296078   
2  34.7     30.567597        31.926581   
3  33.4     28.607036        32.024942   
4  36.2     27.943524        30.071343   


In [ ]:
X_cube = np.concatenate((X_train, X_train**2, X_train**3),axis=1)
w4 = fit_LinearRegression(X_cube, y_train, bias=True)
y_hat4, mse4 = predict_LinearRegression(w4, X_cube, y_train, bias=True)
print("Regression Error with poly-3 features:", mse4)

Regression Error with poly-3 features: 13.060641842807836


In [ ]:
df = pandas.DataFrame({"y": y_train[:5], "linear features": y_hat2[:5], "poly-3 features": y_hat3[:5]})
pandas.set_option('colheader_justify', 'center')
print(df)

     y   linear features  poly-3 features
0  24.0     30.003843        28.340697   
1  21.6     25.025562        23.296078   
2  34.7     30.567597        31.926581   
3  33.4     28.607036        32.024942   
4  36.2     27.943524        30.071343   


However, It is possible that increasing polynomial dimensions may cause **overfitting**, as shown in the image below.

In [ ]:
Image(url='https://drek4537l1klr.cloudfront.net/serrano/Figures/4-3.png', width=800)